# show the max and min in each emotion

In [55]:
import os
import pandas as pd

emolist = 'anger, disgust, fear, happiness, sadness, surprise'.split(', ')
result = {x:[] for x in emolist}
myfolder = 'image_analysis/Exp_collections/230210Exp13Lighting_sys_500/'

for CURR_EMO in emolist:
#     CURR_EMO = 'anger'
    images = [x for x in os.listdir(os.path.join(myfolder, CURR_EMO)) if 'png' in x]

    for img in images:
        test = os.path.join(myfolder, CURR_EMO, img)
        csvname = test.split('.png')[0] + '_emotion.csv'
        tmp = pd.read_csv(csvname)
        result[CURR_EMO].append(float(tmp[CURR_EMO]))

    print(CURR_EMO)
    print(f'max: {max(result[CURR_EMO])}')
    print(f'min: {min(result[CURR_EMO])}')

anger
max: 0.8908626
min: 0.00022180197
disgust
max: 0.9448169
min: 2.9978316e-06
fear
max: 0.7755312
min: 0.00032007878
happiness
max: 0.88944674
min: 0.00030392586
sadness
max: 0.92630714
min: 0.0006400511
surprise
max: 0.9870663
min: 0.00094844046


In [ ]:
# plot for each emotions

In [54]:
result

{'anger': [0.56476116, 0.45512825, 0.0053509707, 0.012822615, 0.55845755],
 'disgust': [6.644751e-06, 0.05863482, 0.78763676, 0.0036472667, 0.8512238],
 'fear': [0.51877135, 0.03438015, 0.0143282255, 0.016819654, 0.00056438893],
 'happiness': [0.05496557, 0.010495846, 0.64524376, 0.71323854, 0.10639376],
 'sadness': [0.671066, 0.014642206, 0.80606645, 0.76308423, 0.7435],
 'surprise': [0.9768141, 0.8902508, 0.9434955, 0.8905662, 0.843911]}

In [41]:
float(tmp[CURR_EMO])

0.56476116

# Model range 

In [60]:
from intensityNet import * 
global intensityModel
intensityModel = ''

def intensityNet_analysis(img, target_emotion, is_save_csv=True):
    # remember to set it before doing analysis

    global intensityModel
    # use intensityModel to detect emo
    detection_res = intensityModel.detect_emo(Image.open(img))
    detection_res = detection_res.tolist()
    output = detection_res[get_target(target_emotion)]

#     # create a pd dataframe
#     detection_res = pd.DataFrame([detection_res], columns=["angry", "disgust", "fear", "happy", "sad", "surprise", "neutral"])
#     # Create DataFrame

#     if is_save_csv:
#         csv_emotion_name = img[:-4]+"_intensitynet.csv"
#         detection_res.to_csv(csv_emotion_name)

    # result = tmp_res[target_emotion]
    return output

def setIntensityModel(target_emotion):
    global intensityModel
    # Load the model
    # ['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise']
    print("target_emotion:", target_emotion)
    if target_emotion.lower() in ["anger", 'angry']:
        model_path = "new_models/angry_fold3_epoch7.pt"
    elif target_emotion.lower() in ["disgust"]:
        model_path = "new_models/disgust_fold3_epoch7.pt"
    elif target_emotion.lower() in ["fear"]:
        model_path = "new_models/fear_fold3_epoch6.pt"
    elif target_emotion.lower() in ["happiness", "happy"]:
        model_path = "new_models/happy_fold2_epoch7.pt"
    elif target_emotion.lower() in ["sadness", "sad"]:
        model_path = "new_models/sad_fold2_epoch6.pt"
    elif target_emotion.lower() in ["surprise"]:
        model_path = "new_models/surprise_fold3_epoch5.pt"
    else:
        model_path = ""

    intensityModel = IntensityNet_type1(model_path)
    return 1

# intensityNet_analysis()


target_emotion: anger


1

In [65]:
emolist = 'anger, disgust, fear, happiness, sadness, surprise'.split(', ')
result = {x:[] for x in emolist}
myfolder = 'image_analysis/Exp_collections/230210Exp13Lighting_sys_500/'


# CURR_EMO = 'anger'

global intensityModel
intensityModel = ''



for CURR_EMO in emolist:
    setIntensityModel(CURR_EMO)
    images = [x for x in os.listdir(os.path.join(myfolder, CURR_EMO)) if 'png' in x]
    
    for i in images:
        result[CURR_EMO].append(intensityNet_analysis(os.path.join(myfolder, CURR_EMO, images[0]), CURR_EMO))
        
    print(CURR_EMO)
    print(len(result[CURR_EMO]))
    print(f'max: {max(result[CURR_EMO])}')
    print(f'min: {min(result[CURR_EMO])}')

    

target_emotion: anger
anger
500
max: 0.6442480683326721
min: 0.390322208404541
target_emotion: disgust
disgust
500
max: 0.6250965595245361
min: 0.35516366362571716
target_emotion: fear
fear
500
max: 0.4678744375705719
min: 0.19767922163009644
target_emotion: happiness
happiness
500
max: 0.5652247071266174
min: 0.2887384295463562
target_emotion: sadness
sadness
500
max: 0.6272915601730347
min: 0.33632296323776245
target_emotion: surprise
surprise
500
max: 0.45335686206817627
min: 0.23851583898067474


In [ ]:
# Test the mixed model

# Function to calculate the mixed output with nonlinear transition using a sigmoid function
def calculate_output_nonlinear(A, B, threshold=0.75, alpha=0.8, B_min=0.39, B_max=0.64, output_min=0.75, output_max=1.2, k=10):
    """
    Calculate the mixed output based on values of A and B, with a threshold-based decision rule.
    
    Parameters:
    A (float): Value from distribution A.
    B (float): Value from distribution B.
    threshold (float): Threshold for A to determine when to mix. Default is 0.75.
    alpha (float): Weighting factor for B' in the mix when A > threshold. Default is 0.8.
    B_min (float): Minimum value of B's original range. Default is 0.39.
    B_max (float): Maximum value of B's original range. Default is 0.64.
    output_min (float): Minimum value of the target range for B' (after scaling). Default is 0.75.
    output_max (float): Maximum value of the target range for B' (after scaling). Default is 1.2.

    Returns:
    float: Final output value based on A and B.
    """
        
    # If A is below or equal to the threshold, output A directly
    if A <= threshold:
        return A
    
    # Scale B to fit within the desired range [output_min, output_max]
    B_mapped = output_min + (B - B_min) * (output_max - output_min) / (B_max - B_min)
    
    # Apply a sigmoid-based weight for smooth transition
    weight = 1 / (1 + np.exp(-k * (A - threshold)))  # Sigmoid function for smoother blending
    
    # Calculate the smooth nonlinear mixed output
    output = weight * (alpha * B_mapped + (1 - alpha) * A) + (1 - weight) * A
    
    return output





## Test the mixed model on old dataset

# extract probe from past experiment

In [ ]:
import numpy as np 
def omega(x):
    return 1.0 / (1 + np.exp(-k * ( x - threshold))

# difference between pyfeat value and resmasknet value (DONE)

In [ ]:
# pyfeat value
from feat import Detector
import os
import cv2
from SiameseRankNet import *
from resmasknet import *
from torchvision.transforms import transforms

detector = Detector(emotion_model = "resmasknet", landmark_model='mobilefacenet')

def get_face_box(start_x, start_y, end_x, end_y):
    center_x, center_y = (start_x + end_x) // 2, (start_y + end_y) // 2
    square_length = ((end_x - start_x) + (end_y - start_y)) // 2 // 2
    square_length *= 1.1
    start_x = int(center_x - square_length)
    start_y = int(center_y - square_length)
    end_x = int(center_x + square_length)
    end_y = int(center_y + square_length)
    return start_x, start_y, end_x, end_y


result = {}
myfolder = 'image_analysis/Exp_collections/230210Exp13Lighting_sys_500/'
CURR_EMO = 'anger'
images = [x for x in os.listdir(os.path.join(myfolder, CURR_EMO)) if 'png' in x]
image_size = (224, 224)
transform = transforms.Compose(
            [transforms.ToPILImage(), transforms.ToTensor()]
        )
model = ResMaskNet()
for i in images[:1]:
    print(i)
    test = os.path.join(myfolder, CURR_EMO, i)
    res = detector.detect_image(test)
    facebox = res.iloc[:,1:5]
    emotions = res.iloc[:, -8:]
    print(f'use pyfeat:\n{[round(x, 5) for x in np.array(emotions.iloc[:, :]).tolist()[0][:-1]]}')
    frame = cv2.imread(test)
    face = detector.detect_faces(frame)
    emotion = model.detect_emo(frame=frame, detected_face=face)
    print(f'use model directly:\n{list(round(x, 5) for x in emotion[0].tolist())}')
    print()
    